In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from model import UNet

device = torch.device("cuda")
root = '/home/hzh/imagesets'
trainset = datasets.VOCSegmentation(root,transform=transforms.Compose([
                           transforms.Resize((500,375)),
                           transforms.ToTensor()                           
                       ]),target_transform=transforms.Compose([
                           transforms.Resize((500,375)),
                           transforms.ToTensor()]))
    
train_loader = torch.utils.data.DataLoader(
        trainset,
        batch_size=2, shuffle=True,num_workers=0)
testset = datasets.VOCSegmentation(root,image_set='val',
                           transform=transforms.Compose([
                           transforms.Resize((500,375)),
                           transforms.ToTensor()                           
                       ]),target_transform=transforms.Compose([
                           transforms.Resize((500,375)),
                           transforms.ToTensor()]))
test_loader = torch.utils.data.DataLoader(
        testset,
        batch_size=2, shuffle=False,num_workers=0)

In [2]:
device = torch.device("cuda")
model = UNet(1, depth=5, up_mode='upsample').to(device)
optimizer = optim.SGD(model.parameters(),
                          lr=0.01,
                          momentum=0.9,
                          weight_decay=0.0005)


def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        output,target=output.view(-1),target.view(-1)
        critizen = nn.BCELoss()
        loss = critizen(output,target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            output,target=output.view(-1),target.view(-1)
            critizen = nn.BCELoss()
            loss = critizen(output,target)
            test_loss += loss.item() # sum up batch loss
            
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}\n'.format(test_loss))

/home/hzh/github/218Unettest/model.py:217: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(m.weight)
/home/hzh/github/218Unettest/model.py:218: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(m.bias, 0)


In [4]:
for epoch in range(100):
    train(model, device, train_loader,optimizer, epoch)
    test(model, device, test_loader)
    if epoch % 10 == 0:
        print('save checkpoint')
        save_name = 'UNet_checkpoint'+str(epoch)+".pt"
        torch.save(model.state_dict(),save_name)

Train Epoch: 0 [0/1464 (0%)]	Loss: 0.247483
Train Epoch: 0 [200/1464 (14%)]	Loss: 0.160033
Train Epoch: 0 [400/1464 (27%)]	Loss: 0.227849
Train Epoch: 0 [600/1464 (41%)]	Loss: 0.244312
Train Epoch: 0 [800/1464 (55%)]	Loss: 0.319642
Train Epoch: 0 [1000/1464 (68%)]	Loss: 0.225794
Train Epoch: 0 [1200/1464 (82%)]	Loss: 0.172319
Train Epoch: 0 [1400/1464 (96%)]	Loss: 0.213757

Test set: Average loss: 0.1204

save checkpoint
Train Epoch: 1 [0/1464 (0%)]	Loss: 0.224408
Train Epoch: 1 [200/1464 (14%)]	Loss: 0.225041
Train Epoch: 1 [400/1464 (27%)]	Loss: 0.164907
Train Epoch: 1 [600/1464 (41%)]	Loss: 0.252075
Train Epoch: 1 [800/1464 (55%)]	Loss: 0.201620
Train Epoch: 1 [1000/1464 (68%)]	Loss: 0.228325
Train Epoch: 1 [1200/1464 (82%)]	Loss: 0.296118
Train Epoch: 1 [1400/1464 (96%)]	Loss: 0.187229

Test set: Average loss: 0.1188

Train Epoch: 2 [0/1464 (0%)]	Loss: 0.171536
Train Epoch: 2 [200/1464 (14%)]	Loss: 0.282337
Train Epoch: 2 [400/1464 (27%)]	Loss: 0.169811
Train Epoch: 2 [600/1464 (41

KeyboardInterrupt: 